## add gzoltar dependencies to pom.xml

In [ ]:
def add_gzoltar(xml):
    # parser for handling xml pre-processing error (e.g., bad xml file)
    parser = etree.XMLParser(recover=True)
    tree = etree.parse(xml, parser=parser)
    root = tree.getroot()
    
    # remove namespace
    for elem in root.getiterator():
        if not hasattr(elem.tag, 'find'): continue
        i = elem.tag.find('}')
        if i >= 0:
            elem.tag = elem.tag[i+1:]
    objectify.deannotate(root, cleanup_namespaces=True)
    
    # add tags
    add_gzoltar_groupid(root, xml)
    add_sbfl_profile(root, xml)
    
    etree.ElementTree(root).write(xml, pretty_print=True, encoding="UTF-8", xml_declaration=True)
            
def add_gzoltar_groupid(root, xml):
    has_gzoltar_groupid = False

    # do for every <build> tag
    for e in root:
        tag = e.tag
        
        # look for build tag
        if tag != 'build':
            continue
            
        # find plugins
        plugins = e.find('plugins')
        if not plugins:
            continue

        # check if <groupId>com.gzoltar</groupId> already exists
        groupId_list = plugins.xpath('./plugin/groupId')
        for gi in groupId_list:
            if gi.text == 'com.gzoltar':
                has_gzoltar_groupid = True
                break

        # if not exists, add <plugins>
        if has_gzoltar_groupid:
            break
        else:
            add_gzoltar_plugin(plugins)
            has_gzoltar_groupid = True
            break
            
    if not has_gzoltar_groupid:
        print('\tadding new <build> tag')
        build = etree.SubElement(root, 'build')
        plugins = etree.SubElement(build, 'plugins')
        add_gzoltar_plugin(plugins)
    
def add_gzoltar_plugin(plugins):
    plugin = etree.SubElement(plugins, 'plugin')

    groupId = etree.SubElement(plugin, 'groupId')
    groupId.text = 'com.gzoltar'

    artifactId = etree.SubElement(plugin, 'artifactId')
    artifactId.text= 'com.gzoltar.maven'

    version = etree.SubElement(plugin, 'version')
    version.text = '1.7.2'

    executions = etree.SubElement(plugin, 'executions')
    execution_goals = etree.SubElement(executions, 'execution')
    
    ids = etree.SubElement(execution_goals, 'id')
    ids.text = 'post-unit-test'
    
    phase = etree.SubElement(execution_goals, 'phase')
    phase.text = 'test'
    
    goals = etree.SubElement(execution_goals, 'goals')
    goal = etree.SubElement(goals, 'goal')
    goal.text = 'fl-report'
    

## add sbfl profile

In [ ]:
def add_profile(profiles):
    profile = etree.SubElement(profiles, 'profile')

    pid = etree.SubElement(profile, 'id')
    pid.text = 'sbfl'

    activation = etree.SubElement(profile, 'activation')
    pproperty = etree.SubElement(activation, 'property')
    p_name = etree.SubElement(pproperty, 'name')
    p_name.text = 'sbfl'

    build = etree.SubElement(profile, 'build')
    plugins = etree.SubElement(build, 'plugins')
    
    plugin_gzoltar = etree.SubElement(plugins, 'plugin')
    
    groupId_gzoltar = etree.SubElement(plugin_gzoltar, 'groupId')
    groupId_gzoltar.text = 'com.gzoltar'
    artifactId_gzoltar = etree.SubElement(plugin_gzoltar, 'artifactId')
    artifactId_gzoltar.text = 'com.gzoltar.maven'
    version_gzoltar = etree.SubElement(plugin_gzoltar, 'version')
    version_gzoltar.text = '1.7.2'
    
    plugin_surefire = etree.SubElement(plugins, 'plugin')
    
    groupId_surefire = etree.SubElement(plugin_surefire, 'groupId')
    groupId_surefire.text = 'org.apache.maven.plugins'
    artifactId_surefire = etree.SubElement(plugin_surefire, 'artifactId')
    artifactId_surefire.text = 'maven-surefire-plugin'
    version_surefire = etree.SubElement(plugin_surefire, 'version')
    version_surefire.text = '3.0.0-M3'
    
    dependencies = etree.SubElement(plugin_surefire, 'dependencies')
    dependency = etree.SubElement(dependencies, 'dependency')
    groupId_junit = etree.SubElement(dependency, 'groupId')
    groupId_junit.text = 'org.apache.maven.surefire'
    artifactId_junit = etree.SubElement(dependency, 'artifactId')
    artifactId_junit.text = 'surefire-junit47'
    version_junit = etree.SubElement(dependency, 'version')
    version_junit.text = '3.0.0-M3'
    
    configuration = etree.SubElement(plugin_surefire, 'configuration')
    testFailureIgnore = etree.SubElement(configuration, 'testFailureIgnore')
    testFailureIgnore.text = 'true'
    systemPropertyVariables = etree.SubElement(configuration, 'systemPropertyVariables')
    gzoltar_agent = etree.SubElement(systemPropertyVariables, 'gzoltar-agent.destfile')
    gzoltar_agent.text = '${project.build.directory}/gzoltar.ser'
    configuration_properties = etree.SubElement(configuration, 'properties')
    configuration_property = etree.SubElement(configuration_properties, 'property')
    configuration_property_name = etree.SubElement(configuration_property, 'name')
    configuration_property_name.text = 'listener'
    configuration_property_value = etree.SubElement(configuration_property, 'value')
    configuration_property_value.text = 'com.gzoltar.internal.core.listeners.JUnitListener'
    

def add_sbfl_profile(root, xml):
    has_sbfl_profile = False
   
    profiles = root.find('profiles')
    
    if not profiles:
        profiles = etree.SubElement(root, 'profiles')
        add_profile(profiles)
    else:
        pid_list = profiles.xpath('./profile/id')
        for pid in pid_list:
            if pid.text == 'sbfl':
                has_sbfl_profile = True
                break
        if not has_sbfl_profile:
            add_profile(profiles)

## perform analysis on formula rankings

In [ ]:
def objectify_name(a_df):
    name_regex = "(?P<package>.*)\$(?P<class>.*)\#(?P<method>.*)\("
    name_dict = re.match(name_regex, a_df['name']).groupdict()
    if not name_dict:
        return None
    a_df['package'] = name_dict['package']
    a_df['class'] = name_dict['class']
    a_df['method'] = name_dict['method']
    return a_df
